In [4]:
import json
import os
import cv2
import torch
import pathlib
import ultralytics
from ultralytics import YOLO
import threading
import customtkinter as ctk
from tkinter import filedialog
from PIL import Image, ImageTk
from datetime import datetime
import numpy as np
import logging

# Initialize logging
logging.basicConfig(level=logging.DEBUG)
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath
os.chdir("C:\\Users\\jadon\\Desktop\\total folder")






# Main window
root = ctk.CTk()
root.geometry("1500x800")
root.title("PCB DEFECT DETECTION")

# Camera and image processing
camera_active = [False, False]  # List to manage multiple cameras
capture_threads = [None, None]  # List to store threads for each camera
selected_camera = ctk.IntVar(value=0)
output_folder = "transformed"
os.makedirs(output_folder, exist_ok=True)

# Initialize grid
PCB_WIDTH = 10000
PCB_HEIGHT = 10000
GRID_SIZE = 2000
NUM_GRIDS_X = PCB_WIDTH // GRID_SIZE
NUM_GRIDS_Y = PCB_HEIGHT // GRID_SIZE
TOTAL_GRIDS = NUM_GRIDS_X * NUM_GRIDS_Y
grid_status = [[0 for _ in range(NUM_GRIDS_X)] for _ in range(NUM_GRIDS_Y)]



weights_path = r"D:\\yolov5default.pt"
data_yaml = r"D:\\pcbprojecta\\BACKUP\\dataset.yaml"

def load_yolo_model(weights_path, data_yaml):
    model = torch.hub.load('yolov5', 'custom', path=weights_path, source='local', force_reload=True)
    model.data = data_yaml
    return model

model = load_yolo_model(weights_path, data_yaml)


# Functions
def start_camera(camera_id):
    """Start camera for the given camera_id."""
    global capture_threads
    if not camera_active[camera_id]:
        capture_threads[camera_id] = threading.Thread(target=process_and_display_frame, args=(camera_id,))
        capture_threads[camera_id].start()

def stop_camera(camera_id):
    """Stop camera for the given camera_id."""
    global camera_active
    camera_active[camera_id] = False
    print(f"Camera {camera_id} feed stopped.")

def process_and_display_frame(camera_id):
    """Process the camera feed and display it."""
    global camera_active
    camera_active[camera_id] = True
    cap = cv2.VideoCapture(camera_id)

    while camera_active[camera_id]:
        ret, frame = cap.read()
        if not ret:
            break

        # Process the frame (e.g., YOLO detection or other processing)
        # You can add defect detection logic here if needed

        # Display the frame in the GUI
        results = model(frame)  
        ram = results.render()[0]
        frame_resized = cv2.resize(ram, (300, 300))
        frame_pil = Image.fromarray(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
        frame_tk = ImageTk.PhotoImage(frame_pil)

        if camera_id == 0:
            camera_0_label.configure(image=frame_tk)
            camera_0_label.image = frame_tk
        if camera_id == 1:
            camera_1_label.configure(image=frame_tk)
            camera_1_label.image = frame_tk        
        if camera_id == 2:
            camera_2_label.configure(image=frame_tk)
            camera_2_label.image = frame_tk        
        if camera_id == 3:
            camera_3_label.configure(image=frame_tk)
            camera_3_label.image = frame_tk
        elif camera_id == 4:
            camera_4_label.configure(image=frame_tk)
            camera_4_label.image = frame_tk

    cap.release()

def save_image(image, save_path):
    """General function to save images."""
    if image is not None:
        cv2.imwrite(save_path, image)
        print(f"Image saved at {save_path}")

def display_defect_type(results):
    """Display the type of defect detected."""
    defect_types = [result['name'] for result in results.pandas().xyxy[0].to_dict(orient="records")]
    defect_text = ", ".join(defect_types) if defect_types else "No defects detected"
    defect_label.configure(text=f"Defects: {defect_text}")

def update_composite_image():
    """Update composite image from the grid status."""
    composite_width = NUM_GRIDS_X * GRID_SIZE
    composite_height = NUM_GRIDS_Y * GRID_SIZE
    composite_img = np.zeros((composite_height, composite_width, 3), dtype=np.uint8)

    for y in range(NUM_GRIDS_Y):
        for x in range(NUM_GRIDS_X):
            if grid_status[y][x] == 1:
                grid_filename = os.path.join(output_folder, f"grid_{y}_{x}.png")
                grid_img = cv2.imread(grid_filename)
                if grid_img is not None:
                    grid_resized = cv2.resize(grid_img, (GRID_SIZE, GRID_SIZE))
                    y_start = y * GRID_SIZE
                    x_start = x * GRID_SIZE
                    composite_img[y_start:y_start + GRID_SIZE, x_start:x_start + GRID_SIZE] = grid_resized

    # Draw grid lines (red boundaries)
    for y in range(NUM_GRIDS_Y + 1):
        cv2.line(composite_img, (0, y * GRID_SIZE), (composite_width, y * GRID_SIZE), (0, 0, 255), 1)
    for x in range(NUM_GRIDS_X + 1):
        cv2.line(composite_img, (x * GRID_SIZE, 0), (x * GRID_SIZE, composite_height), (0, 0, 255), 1)

    # Save and display composite image
    cv2.imwrite("composite_image.png", composite_img)
    resized_frame = cv2.resize(composite_img, (400, 400))
    composite_pil_img = Image.fromarray(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB))
    composite_tk_img = ImageTk.PhotoImage(composite_pil_img)
    composite_grid_label.configure(image=composite_tk_img, text="")
    composite_grid_label.image = composite_tk_img

# GUI Elements
controls_frame = ctk.CTkFrame(root)
controls_frame.pack(side="right", fill="y", padx=10, pady=10)

# Buttons
start_button_0 = ctk.CTkButton(controls_frame, text="Start Camera 0", command=lambda: start_camera(0))
start_button_0.pack(pady=10)
stop_button_0 = ctk.CTkButton(controls_frame, text="Stop Camera 0", command=lambda: stop_camera(0))
stop_button_0.pack(pady=10)

start_button_1 = ctk.CTkButton(controls_frame, text="Start Camera 1", command=lambda: start_camera(1))
start_button_1.pack(pady=10)
stop_button_1 = ctk.CTkButton(controls_frame, text="Stop Camera 1", command=lambda: stop_camera(1))
stop_button_1.pack(pady=10)

start_button_2 = ctk.CTkButton(controls_frame, text="Start Camera 2", command=lambda: start_camera(2))
start_button_2.pack(pady=10)
stop_button_2 = ctk.CTkButton(controls_frame, text="Stop Camera 2", command=lambda: stop_camera(2))
stop_button_2.pack(pady=10)

start_button_3 = ctk.CTkButton(controls_frame, text="Start Camera 3", command=lambda: start_camera(3))
start_button_3.pack(pady=10)
stop_button_3 = ctk.CTkButton(controls_frame, text="Stop Camera 3", command=lambda: stop_camera(3))
stop_button_3.pack(pady=10)

start_button_4 = ctk.CTkButton(controls_frame, text="Start Camera 4", command=lambda: start_camera(4))
start_button_4.pack(pady=10)
stop_button_4 = ctk.CTkButton(controls_frame, text="Stop Camera 4", command=lambda: stop_camera(4))
stop_button_4.pack(pady=10)

# Camera feed labels
camera_0_label = ctk.CTkLabel(root, text="Camera 0", width=200, height=200)
camera_0_label.pack(side="left", padx=10, pady=10, anchor="nw")  # Bottom-left corner

camera_1_label = ctk.CTkLabel(root, text="Camera 1", width=200, height=200)
camera_1_label.pack(side="left", padx=10, pady=10, anchor="n")  # Top-left corner

camera_2_label = ctk.CTkLabel(root, text="Camera 2", width=200, height=200)
camera_2_label.pack(side="left", padx=10, pady=10, anchor="ne")  # Top-right corner

camera_3_label = ctk.CTkLabel(root, text="Camera 3", width=200, height=200)
camera_3_label.pack(side="left", padx=10, pady=10, anchor="ne")  # Bottom-right corner

camera_4_label = ctk.CTkLabel(root, text="Camera 4", width=200, height=200)
camera_4_label.pack(side="left", padx=10, pady=10, anchor="ne") 
# Defect label
defect_label = ctk.CTkLabel(root, text="Defects: ", font=("Arial", 16))
defect_label.pack(side="top", pady=10)

# Start the GUI loop
root.mainloop()

YOLOv5  2024-12-11 Python-3.12.8 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\jadon\Desktop\total folder\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\jadon\Desktop\total folder\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\jadon\Desktop\total folder\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\jadon\Desktop\total folder\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)`

In [3]:
pip install ultralytics

  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached torchvision-0.20.1-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/913.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/913.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/913.6 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/913.6 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/913.6 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/913.6 kB ? eta -:--:--
   ----------- -------